In [1]:
%matplotlib inline

In [2]:
# stdlib imports
import os.path

# gmprocess imports
from gmprocess.io.asdf.stream_workspace import StreamWorkspace
from gmprocess.io.test_utils import read_data_dir
from gmprocess.io.read import read_data
from gmprocess.streamcollection import StreamCollection
from gmprocess.processing import process_streams
from gmprocess.event import get_event_object

Methods of finding and reading strong motion data will vary, the function below automates to a certain extent
the selection of test data from the repository. Since each data directory contains an "event.json" file, this function also returns an event dictionary, required for the processing method.

In [3]:
datafiles, origin = read_data_dir('knet', 'us2000cnnl')

The read_data below finds the appropriate data reader for the format supplied.

In [4]:
tstreams = []
for dfile in datafiles:
    tstreams += read_data(dfile)

In [5]:
len(tstreams)

27

Create a StreamCollection from the list of KNET streams. This class organizes the data into (nominally) 3-channel Streams (one per station), and does some other validation as well. This object is the required input for the process_streams method.

In [6]:
instreams = StreamCollection(tstreams)
outstreams = process_streams(instreams, origin)
print(len(outstreams))

9


Now let's create a workspace (ASDF file) to store these streams and then some associated metadata.

In [8]:
outfile = os.path.join(os.path.expanduser('~'), 'knet_test.hdf')
if os.path.isfile(outfile):
    os.remove(outfile)
workspace = StreamWorkspace(outfile)
# create an Obspy event object from our dictionary
event = get_event_object(origin)

# add the "raw" (KNET actually pre-converts to gals) data
workspace.addStreams(event, instreams, label='rawknet')
eventid = origin['id']

workspace.addStreams(event, outstreams, label='processed')
processed_tags = workspace.getStreamTags(eventid, label='processed')

In [9]:
raw_tags

['aom001_rawknet',
 'aom002_rawknet',
 'aom003_rawknet',
 'aom004_rawknet',
 'aom005_rawknet',
 'aom006_rawknet',
 'aom007_rawknet',
 'aom008_rawknet',
 'aom009_rawknet']

In [10]:
processed_tags

['aom001_processed',
 'aom002_processed',
 'aom003_processed',
 'aom004_processed',
 'aom005_processed',
 'aom006_processed',
 'aom007_processed',
 'aom008_processed',
 'aom009_processed']